# Proyecto SQL

# Conexión a la Base de Datos

In [10]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],

db_config['pwd'],

db_config['host'],
db_config['port'],

db_config['db'])
# Conectarse a la base de datos 2
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Objetivo del estudio

El objetivo de este análisis es examinar la base de datos de un servicio de lectura en línea para comprender las tendencias en la publicación y consumo de libros. A partir de esta información, buscamos responder preguntas clave sobre la popularidad de los libros, el comportamiento de los usuarios y el desempeño de las editoriales y autores.

Los resultados ayudarán a generar una propuesta de valor para un nuevo producto en el mercado de aplicaciones de lectura, optimizando su catálogo y estrategias de recomendación.

## Exploración de la tablas

In [22]:
# Definimos una función para mostrar la tabla
def show_table_preview(table_name):
    query = f"SELECT * FROM {table_name} LIMIT 5;"
    df = pd.read_sql(query, con=engine)
    print(f"\nTabla: {table_name}")
    print(df.head())

show_table_preview("books")
show_table_preview("authors")
show_table_preview("publishers")
show_table_preview("ratings")
show_table_preview("reviews")

# Ejecución del query
def execute_query(query):
    return pd.read_sql(query, con=engine)


Tabla: books
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  

Tabla: authors
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5

## Encuentra el número de libros publicados después del 1 de enero de 2000.

In [23]:
query = """
SELECT COUNT(*) AS num_books
FROM books
WHERE publication_date > '2000-01-01';
"""
print("\nNúmero de libros publicados después del 1 de enero de 2000:")
pd.read_sql(query, con=engine)


Número de libros publicados después del 1 de enero de 2000:


num_books
0        819

## Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [24]:
query = """
SELECT 
    books.book_id, 
    books.title, 
    COUNT(reviews.review_id) AS num_reviews,
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id, books.title
ORDER BY num_reviews DESC;
"""
print("\nNúmero de reseñas y calificación promedio por libro:")
pd.read_sql(query, con=engine)


Número de reseñas y calificación promedio por libro:


book_id                                              title  num_reviews  \
0        948                            Twilight (Twilight  #1)         1120   
1        750                The Hobbit  or There and Back Again          528   
2        673                             The Catcher in the Rye          516   
3        302  Harry Potter and the Prisoner of Azkaban (Harr...          492   
4        299  Harry Potter and the Chamber of Secrets (Harry...          480   
..       ...                                                ...          ...   
995      221                          Essential Tales and Poems            0   
996      808                            The Natural Way to Draw            0   
997      191  Disney's Beauty and the Beast (A Little Golden...            0   
998       83    Anne Rice's The Vampire Lestat: A Graphic Novel            0   
999      672   The Cat in the Hat and Other Dr. Seuss Favorites            0   

     avg_rating  
0          3.66  
1          4.13  
2          3.83  
3          4.41  
4          4.29  
..          ...  
995        4.00  
996        3.00  
997        4.00  
998        3.67  
999        5.00  

[1000 rows x 4 columns]

De este análisis podemos ver que el libro con mayor reviews fue "Twilight", superando por el doble al segundo lugar que fue "The Hobbit", aunque este último tuvo un mejor rating. Además, también vemos que varios libros no tuvieron nada de reseñas.

## Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [25]:
query = """
SELECT 
    publishers.publisher, 
    COUNT(books.book_id) AS num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 1;
"""
print("\nEditorial con mayor número de libros de más de 50 páginas:")
pd.read_sql(query, con=engine)


Editorial con mayor número de libros de más de 50 páginas:


publisher  num_books
0  Penguin Books         42

De este análisis podemos ver que "Penguin Books" es la editorial con mayor número de libros publicados mayores a 50 páginas.

## Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [26]:
query = """
SELECT 
    authors.author, 
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN (
    SELECT book_id 
    FROM ratings 
    GROUP BY book_id 
    HAVING COUNT(rating) >= 50
)
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1;
"""
print("\nAutor con la más alta calificación promedio (mínimo 50 calificaciones):")
pd.read_sql(query, con=engine)


Autor con la más alta calificación promedio (mínimo 50 calificaciones):


author  avg_rating
0  J.K. Rowling/Mary GrandPré        4.29

De este análisis podemos observar que J.K Rowling tiene la calificación más alta en cuanto a calificación con un 4.29.

## Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [27]:
query = """
WITH active_users AS (
    SELECT username 
    FROM ratings 
    GROUP BY username 
    HAVING COUNT(rating) > 50
)
SELECT 
    ROUND(AVG(review_count), 2) AS avg_text_reviews
FROM (
    SELECT COUNT(reviews.review_id) AS review_count
    FROM reviews
    WHERE username IN (SELECT username FROM active_users)
    GROUP BY username
) AS user_reviews;
"""
print("\nPromedio de reseñas de texto entre usuarios que calificaron más de 50 libros:")
pd.read_sql(query, con=engine)


Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:


avg_text_reviews
0             24.33

Del análisis sabemos que el promedio de reseñas de los usuarios que calificaron más de 50 libros fue de 24.33.